In [2]:
from keras.layers import Input, Dense
from keras.models import Model

Using TensorFlow backend.


In [3]:
from keras.datasets import mnist
import numpy as np

In [4]:
(x_train, _), (x_test, _) = mnist.load_data()

In [8]:
##
import torch as T
x2 = T.load("../../91r/mnist/train-images-idx3-ubyte.T").numpy()
(x_train==x2).all()

True

In [8]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [9]:
input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

In [10]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.3330 - val_loss: 0.2635
Epoch 2/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.2563 - val_loss: 0.2485
Epoch 3/100
60000/60000 [==============================] - 3s 56us/step - loss: 0.2345 - val_loss: 0.2210
Epoch 4/100
60000/60000 [==============================] - 3s 55us/step - loss: 0.2136 - val_loss: 0.2050
Epoch 5/100
60000/60000 [==============================] - 3s 57us/step - loss: 0.1998 - val_loss: 0.1945
Epoch 6/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.1917 - val_loss: 0.1859
Epoch 7/100
60000/60000 [==============================] - 4s 59us/step - loss: 0.1829 - val_loss: 0.1766
Epoch 8/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.1749 - val_loss: 0.1692
Epoch 9/100
60000/60000 [==============================] - 4s 60us/step - loss: 0.1682 - val_loss: 0.1